In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [5]:
from django.db.models import Q
from tqdm import tqdm
from edc_constants.constants import NO, NOT_APPLICABLE
from flourish_child.models import ChildPregTesting, ChildTannerStaging
from flourish_caregiver.models import CaregiverChildConsent

UPDATE_FIELDS = ['menarche', 'menarche_start_dt', 'menarche_start_est']
FIELD_MAP = {'menarche': 'manarche_dt_avail', 'menarche_start_dt': 'menarche_dt',
             'menarche_start_est': 'menarche_dt_est'}


def update_preg_testing(source_obj, preg_objs):
    for field in UPDATE_FIELDS:
        for preg_obj in preg_objs:
            source_field_value = getattr(source_obj, FIELD_MAP[field],
                                         getattr(source_obj, field, ''))
            target_field_value = getattr(preg_obj, field)
            if source_field_value != target_field_value and source_field_value is not None:
                setattr(preg_obj, field, source_field_value)
                preg_obj.save()


def process_child_record(child):
    tanner_staging = ChildTannerStaging.objects.filter(
        ~Q(manarche_dt_avail__in=['not_reached', NOT_APPLICABLE, NO]),
        child_visit__subject_identifier=child, )
    preg_objs = ChildPregTesting.objects.filter(child_visit__subject_identifier=child)

    if tanner_staging.exists():
        source_obj = tanner_staging.latest('report_datetime')
        update_preg_testing(source_obj, preg_objs)
    elif preg_objs.exists():
        source_obj = preg_objs.earliest('report_datetime')
        update_preg_testing(source_obj, preg_objs)


def main():
    all_children = CaregiverChildConsent.objects.values_list('subject_identifier',
                                                             flat=True).distinct()
    for child in tqdm(all_children):
        process_child_record(child)


main()

100%|██████████| 1304/1304 [10:24<00:00,  2.09it/s]


In [12]:
unique = list(set(ChildTannerStaging.objects.values_list('manarche_dt_avail',
                                                         flat=True).distinct()))

unique = list(set(ChildPregTesting.objects.values_list('menarche', flat=True).distinct()))

In [13]:
unique

['', 'Yes', 'No']